In [1]:
%load_ext kedro

[10/16/25 20:29:30] INFO     Using                                                                  ]8;id=921977;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=607461;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.1                
                             1/site-packages/kedro/framework/project/rich_logging.yml' as logging                  
                             configuration.                                                                        

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=269474;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=274709;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=375493;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=33568;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=178176;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=644901;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /home/zishan/Documents/projects/commodity-price-forecasting.                          
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/16/25 20:29:30] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=633919;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=210590;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/16/25 20:29:32] INFO     Kedro project commodity_price_forecasting                              ]8;id=606397;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=925195;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=915854;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=903297;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

In [ ]:
df = catalog.load('clean_data')
df.info()

[10/16/25 20:29:55] INFO     Loading data from clean_data (ParquetDataset)...                  ]8;id=131020;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=143161;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/site-packages/kedro/io/data_catalog.py#1046\1046]8;;\

In [9]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import periodogram
from sklearn.preprocessing import StandardScaler
import numpy as np

In [10]:
class CommodityItem:
    def __init__(self, name, df):
        self.name = name
        self.df = df.sort_values('date')
        self.series = self.df.set_index('date')['high_price']
        
    def extract_features(self):
        """Extract time series features for clustering"""
        features = {}
        ts = self.series
        
        features['trend_strength'] = self._get_trend_strength(ts)
        features['trend_slope'] = self._get_trend_slope(ts)
        
        features['seasonality_strength'] = self._get_seasonality_strength(ts)
        features['seasonal_period'] = self._detect_seasonal_period(ts)
        
        features['volatility'] = ts.pct_change().std()
        features['hurst_exponent'] = self._get_hurst_exponent(ts)
        
        features['skewness'] = ts.skew()
        features['kurtosis'] = ts.kurtosis()
        features['cv'] = ts.std() / ts.mean()
        
        return features
    
    def _get_trend_strength(self, ts):
        """Measure trend strength using variance decomposition"""
        try:
            result = seasonal_decompose(ts.dropna(), period=365, extrapolate_trend='freq')
            return np.var(result.trend) / (np.var(result.trend) + np.var(result.resid))
        except:
            return 0
    
    def _get_seasonality_strength(self, ts):
        """Measure seasonality strength"""
        try:
            result = seasonal_decompose(ts.dropna(), period=365, extrapolate_trend='freq')
            return np.var(result.seasonal) / (np.var(result.trend) + np.var(result.seasonal) + np.var(result.resid))
        except:
            return 0
    
    def _get_trend_slope(self, ts):
        """Linear trend slope"""
        x = np.arange(len(ts))
        return np.polyfit(x, ts.values, 1)[0]
    
    def _detect_seasonal_period(self, ts):
        """Detect dominant seasonal period using FFT"""
        f, Pxx = periodogram(ts.dropna())
        if len(Pxx) > 0:
            return 1/f[np.argmax(Pxx[1:]) + 1]
        return 0
    
    def _get_hurst_exponent(self, ts):
        """Hurst exponent for long-term memory/persistence"""
        lags = range(2, 100)
        tau = [np.std(np.subtract(ts[lag:].values, ts[:-lag].values)) for lag in lags]
        return np.polyfit(np.log(lags), np.log(tau), 1)[0]

In [13]:
class CommodityClusterAnalyzer:
    def __init__(self, df):
        self.df = df
        self.items = {}
        self.feature_matrix = None
        self.clusters = None
        
    def create_items(self):
        """Create CommodityItem objects for each unique item"""
        for item_name in self.df['item'].unique():
            item_df = self.df[self.df['item'] == item_name]
            self.items[item_name] = CommodityItem(item_name, item_df)
    
    def extract_all_features(self):
        """Extract features for all items with NaN handling"""
        features_list = []
        item_names = []
        
        for name, item in self.items.items():
            try:
                features = item.extract_features()
                # Convert to list and check for NaN/Inf
                feature_values = list(features.values())
                
                # Check for any NaN or infinite values
                if not any(np.isnan(x) or np.isinf(x) for x in feature_values if isinstance(x, (int, float))):
                    features_list.append(feature_values)
                    item_names.append(name)
                else:
                    print(f"Skipping {name} due to NaN/Inf values")
                    
            except Exception as e:
                print(f"Error processing {name}: {e}")
                continue
        
        self.feature_matrix = np.array(features_list)
        self.feature_names = list(features.keys()) if features_list else []
        self.item_names = item_names
        
        print(f"Successfully processed {len(features_list)} items out of {len(self.items)}")
        return self.feature_matrix
    
    def perform_clustering(self, n_clusters=5):
        """Perform hierarchical clustering with robust error handling"""
        if self.feature_matrix is None or len(self.feature_matrix) == 0:
            raise ValueError("No valid features extracted for clustering")
        
        # Standardize features and handle any remaining NaN/Inf
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(self.feature_matrix)
        
        # Final check for NaN/Inf
        if np.any(~np.isfinite(X_scaled)):
            print("Warning: NaN/Inf values found after scaling. Replacing with 0.")
            X_scaled = np.nan_to_num(X_scaled)
        
        # Hierarchical clustering
        Z = linkage(X_scaled, method='ward')
        self.clusters = fcluster(Z, n_clusters, criterion='maxclust')
        
        return self.clusters, Z

In [15]:

analyzer = CommodityClusterAnalyzer(df)
analyzer.create_items()
analyzer.extract_all_features()

clusters, linkage_matrix = analyzer.perform_clustering(n_clusters=6)

cluster_summary = analyzer.perform_clustering()

for cluster_id, summary in cluster_summary.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Items: {summary['items']}")
    print(f"  Avg Trend: {summary['avg_trend']:.3f}")
    print(f"  Avg Seasonality: {summary['avg_seasonality']:.3f}")
    print(f"  Size: {summary['size']}")
    print()

[10/16/25 20:56:50] WARNING  /home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11 ]8;id=587308;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=516499;file:///home/zishan/anaconda3/envs/commodity_price_forecasting/lib/python3.11/warnings.py#110\110]8;;\
                             /site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid                    
                             value encountered in subtract                                                         
                               sqr = _ensure_numeric((avg - values) ** 2)                                          
                                                                                                                   

Skipping আদা (দেশী) due to NaN/Inf values
Skipping ইলিশ due to NaN/Inf values
Skipping খাসী due to NaN/Inf values
Skipping গরু due to NaN/Inf values
Skipping পাম অয়েল (সুপার) due to NaN/Inf values
Skipping পিঁয়াজ (আমদানি) due to NaN/Inf values
Successfully processed 45 items out of 51


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6                                                                                             │
│    7 cluster_summary = analyzer.perform_clustering()                                             │
│    8                                                                                             │
│ ❱  9 for cluster_id, summary in cluster_summary.items():                                         │
│   10 │   print(f"Cluster {cluster_id}:")                                                         │
│   11 │   print(f"  Items: {summary['items']}")                                                   │
│   12 │   print(f"  Avg Trend: {summary['avg_trend']:.3f}")                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'items'